### You can find other fine tuning guides here:
[RAG With Models on GitHub](https://github.com/mosh98/RAG_With_Models/tree/main)

# Step 1

## Prerequisites

In [1]:
!nvidia-smi

Thu Sep 19 02:22:09 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.99                 Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   40C    P3             19W /  285W |    1116MiB /  12282MiB |     23%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we specify the model ID and then we load it with our previously defined quantization configuration.Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [5]:
# if you are using google colab

# import os
# from google.colab import userdata
# os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

# Step 2 - Model loading
We'll load the model using QLoRA quantization to reduce the usage of memory


In [6]:
model_id = "google/gemma-2-9b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)



Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.10s/it]


In [9]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [8]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=model, tokenizer=tokenizer)
print(result)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  
model


  user
  Below is a description of a problem. Write a response that provides the solution. Be sure to use all the details in the original text.
  The difference of two numbers is 253. The quotient is 14 when the first number is divided by the second. What is the first number.
  code to find two numbers with a difference and quotient of 253 and 14
  model


  u=user
  Below is a short story. Write a brief summary describing its most important details. Include details from the first paragraph, middle, and end of the story.
  My pet fish died so I decided to bury him in my backyard. I scooped out a hole, and carefully put him in. I carefully piled the dirt in, and I made a nice tombstone, and put it at his grave. I loved my pet fish
  A man buried his dead fish in the backyard and marked the grave with a stone

# Step 3 - Load dataset for finetuning

### Lets Load the Dataset



We will be using this [dataset](https://huggingface.co/datasets/TokenBender/code_instructions_122k_alpaca_style) which is curated by [TokenBender (e/xperiments)](https://twitter.com/4evaBehindSOTA) and is an excellent data source for fine-tuning models for code generation. It follows the alpaca style of instructions, which is an excellent starting point for this task. The dataset structure should resemble the following:

```json
{
  "instruction": "Create a function to calculate the sum of a sequence of integers.",
  "input": "[1, 2, 3, 4, 5]",
  "output": "# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}
```

In [10]:
from datasets import load_dataset

dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")
dataset

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Aiden\.cache\huggingface\hub\datasets--TokenBender--code_instructions_122k_alpaca_style. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 121959/121959 [00

Dataset({
    features: ['input', 'text', 'instruction', 'output'],
    num_rows: 121959
})

In [11]:
df = dataset.to_pandas()
df.head(10)

,input,text,instruction,output
0,"[1, 2, 3, 4, 5]",Below is an instruction that describes a task....,Create a function to calculate the sum of a se...,# Python code\ndef sum_sequence(sequence):\n ...
1,"str1 = ""Hello ""\nstr2 = ""world""",Below is an instruction that describes a task....,Develop a function that will add two strings,"def add_strings(str1, str2):\n """"""This func..."
2,,Below is an instruction that describes a task....,Design a data structure in C++ to store inform...,#include <map>\n#include <string>\n\nclass Gro...
3,"[3, 1, 4, 5, 9, 0]",Below is an instruction that describes a task....,Implement a sorting algorithm to sort a given ...,def bubble_sort(arr):\n n = len(arr)\n \n ...
4,Not applicable,Below is an instruction that describes a task....,Design a Swift application for tracking expens...,import UIKit\n\nclass ExpenseViewController: U...
5,Not Applicable,Below is an instruction that describes a task....,Create a REST API to convert a UNIX timestamp ...,<?php\n$timestamp = $_GET['timestamp'];\n\nif(...
6,website: www.example.com \ndata to crawl: phon...,Below is an instruction that describes a task....,Generate a Python code for crawling a website ...,import requests\nimport re\n\ndef crawl_websit...
7,,Below is an instruction that describes a task....,Create a Python list comprehension to get the ...,"[x*x for x in [1, 2, 3, 5, 8, 13]]"
8,,Below is an instruction that describes a task....,Create a MySQL query to find the most expensiv...,SELECT * FROM products ORDER BY price DESC LIM...
9,Not applicable,Below is an instruction that describes a task....,Create a data structure in Java for storing an...,public class Library {\n \n // map of books in...


In [12]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
    # Samples with additional context into.
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} here are the inputs {data_point["input"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""
    return text

In [13]:
# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map: 100%|██████████| 121959/121959 [00:09<00:00, 13209.93 examples/s]


In [14]:
dataset

Dataset({
    features: ['input', 'text', 'instruction', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 121959
})

In [16]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [17]:
print(test_data)

Dataset({
    features: ['input', 'text', 'instruction', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 24392
})


# Step 4 - Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [18]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [19]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [20]:
modules = find_all_linear_names(model)
print(modules)

['up_proj', 'q_proj', 'gate_proj', 'o_proj', 'k_proj', 'v_proj', 'down_proj']


In [21]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [22]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 216072192 | total: 9457778176 | Percentage: 2.2846%


# Step 5 - Run the training!

## With HF

In [23]:
# import transformers

# tokenizer.pad_token = tokenizer.eos_token


# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         warmup_steps=0.03,
#         max_steps=100,
#         learning_rate=2e-4,
#         fp16=True,
#         logging_steps=1,
#         output_dir="outputs_mistral_b_finance_finetuned_test",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )


### Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this tutorial, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [24]:
import transformers

from trl import SFTTrainer


tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()


trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        #warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:407: UserWarning: 

## Lets start training

In [25]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
  1%|          | 1/100 [00:57<1:35:33, 57.91s/it]

{'loss': 2.8496, 'grad_norm': 2.178943157196045, 'learning_rate': 0.00019800000000000002, 'epoch': 0.0}


 Share adapters on the 🤗 Hub

In [ ]:
new_model = "gemma2-Code-Instruct-Finetune-test" #Name of the model you will be pushing to huggingface model hub

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
#merged_model.push_to_hub(new_model, use_temp_dir=False)
#tokenizer.push_to_hub(new_model, use_temp_dir=False)

## Test out Finetuned Model

In [ ]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=merged_model, tokenizer=tokenizer)

In [ ]:
print(result)


  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  
model


  modelimport time
  from functools import lru_cache

  def fibonacci(n: int) -> int:
      # If n is a negative number
      if n < 0:
          print("Incorrect input")
      
      # First fibonacci value is 0
      elif n == 1:
          return 0 # First value is 0

      # Second fibonacci value is 1
      elif n == 2:
          return 1 # Second value is 1
      
      else:
          return fibonacci(n-1) + fibonacci(n-2)


  # Driver Program
  n = 9
  start = time.time()
  print(fibonacci(n))
  end = time.time()

  print("Time taken =",end - start)
  # Output
  55
  0.6109077821960449 
user
  code the fibonacci series in python using dynamic programming
  
  modellimport time
  
  def fibonacci(n):
      
      # If n is 0
      if n == 0:
          return 0
      
      # If n is 1
      elif n == 